In [ ]:
%matplotlib inline
import numpy as np
import jupyter_contrib_nbextensions as jex
import matplotlib as mpl
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
import os,re

import 